In [1]:
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**LOADING DATA**

In [2]:
# Loading Data
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [3]:
print(reviews[:2000])
print()
print(labels[:2000])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

**CLEARING DATA**

In [4]:
from string import punctuation

print(punctuation)
# Clearing data

reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

words = all_text.split()
print(words[:40])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'high', 's', 'satire', 'is', 'much', 'closer']


**ENCODING WORDS**

In [6]:
from collections import Counter 

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)}

reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [7]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [8]:
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

In [10]:
print(encoded_labels[:10])

[1 0 1 0 1 0 1 0 1 0]


**REMOVING OUTLINERS**

In [11]:
review_lens = Counter([len(x) for x in reviews_ints])

In [12]:
print('Zero length reviews : {}'.format(review_lens[0]))
print('Max review Length: {}'.format(max(review_lens)))

Zero length reviews : 1
Max review Length: 2514


In [49]:
print('Number of reviews before : {}'.format(len(reviews_ints)))

# Removing 0 length reviews 
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review)!=0]

reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews After : {}'.format(len(reviews_ints)))

Number of reviews before : 25000
Number of reviews After : 25000


**PADDING SEQUENCES**

In [50]:
def pad_features(reviews_ints, seq_length):
    
    features = np.zeros((len(reviews_ints), seq_length) , dtype=int)
    
    for i, review in enumerate(reviews_ints):
        features[i, -len(review):] = np.array(review)[:seq_length]
        
    return features

In [51]:
# Testing

seq_len = 200

features = pad_features(reviews_ints, seq_len)

assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# Printing 3 rows and 10 columns
print(features[:3, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]]


**TRAINING, VALIDATION and TEST**

In [52]:
split_frac = 0.8

split_idx = int(len(features) * split_frac)

train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

valid_idx = int(len(remaining_x) * 0.5)

valid_x, test_x = remaining_x[:valid_idx], remaining_x[valid_idx:]
valid_y, test_y = remaining_y[:valid_idx], remaining_y[valid_idx:]

print('\t\t\tFeature Shapes :')
print('Train Shape: \t\t{}'.format(train_x.shape))
print('Test Shape: \t\t{}'.format(test_x.shape))
print('Valid Shape: \t\t{}'.format(valid_x.shape))

			Feature Shapes :
Train Shape: 		(20000, 200)
Test Shape: 		(2500, 200)
Valid Shape: 		(2500, 200)


**DATA LOADERS AND BATCHING**

In [65]:
# TensorDatset creates a dataset to be fed to the DataLoader
from torch.utils.data import TensorDataset, DataLoader

train_set = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_set = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
valid_set = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))

batch_size = 50

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

In [55]:
# Obtaining one set of training set

data = iter(training_loader)
x, y = data.next()

print('Sample input size: ', x.size()) # batch_size, seq_length
print('Sample input: \n', x)
print()
print('Sample label size: ', y.size()) # batch_size
print('Sample label: \n', y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,   138,  2556,  1907],
        [ 2471,  5067, 19717,  ...,    67,  2750,     7],
        [    8,    13,    77,  ...,    37,   109,  1479],
        ...,
        [   10,    89,    23,  ...,   825,    14,   605],
        [    0,     0,     0,  ...,   789,     8,    45],
        [ 7935,  7364,     6,  ...,  5432,  7109,   790]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        1, 1])


**CREATING MODEL**

In [72]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.sig(self.fc(out))
        
        # reshape to have batch_size first
        
        out = out.view(batch_size, -1)
        out = out[:,-1] # last batch of labels
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                 weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        
        return hidden
        
        
        

**Instantiate Network**

In [73]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


**TRAINING**

In [74]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [78]:
epochs = 4

counter = 0
print_every = 100
clip = 5 #Gradient Clipping

net.to(device)
net.train()

for e in range(epochs):
    # Initializing hidden layer
    h = net.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        
        net.zero_grad()
        
        output, h = net(inputs, h)
        
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        # Loss Stats
        
        if counter % print_every == 0:
            # Validation Loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            
            for inputs, labels in valid_loader:
                
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])
                
                output, val_h = net(inputs, val_h)
                
                val_loss = criterion(output.squeeze(), labels.float())
                
                val_losses.append(val_loss.item())
                
            net.train()
            
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.452300... Val Loss: 0.626118
Epoch: 1/4... Step: 200... Loss: 0.478042... Val Loss: 0.544194
Epoch: 1/4... Step: 300... Loss: 0.641316... Val Loss: 0.621200
Epoch: 1/4... Step: 400... Loss: 0.545369... Val Loss: 0.464491
Epoch: 2/4... Step: 500... Loss: 0.439087... Val Loss: 0.460830
Epoch: 2/4... Step: 600... Loss: 0.362222... Val Loss: 0.437105
Epoch: 2/4... Step: 700... Loss: 0.453288... Val Loss: 0.481985
Epoch: 2/4... Step: 800... Loss: 0.302630... Val Loss: 0.451412
Epoch: 3/4... Step: 900... Loss: 0.344910... Val Loss: 0.499983
Epoch: 3/4... Step: 1000... Loss: 0.197069... Val Loss: 0.473945
Epoch: 3/4... Step: 1100... Loss: 0.186173... Val Loss: 0.526651
Epoch: 3/4... Step: 1200... Loss: 0.210928... Val Loss: 0.434383
Epoch: 4/4... Step: 1300... Loss: 0.096828... Val Loss: 0.521403
Epoch: 4/4... Step: 1400... Loss: 0.299026... Val Loss: 0.571656
Epoch: 4/4... Step: 1500... Loss: 0.101120... Val Loss: 0.538460
Epoch: 4/4... Step: 1600... Loss: 

**Testing**

In [83]:
test_losses = []
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:
    
    inputs, labels = inputs.to(device), labels.to(device)
    
    h = tuple([each.data for each in h])
    
    output, h = net(inputs,h)
    
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.557
Test accuracy: 0.796


**INFERENCE**

In [79]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [85]:
def tokenize_review(review):
    review = review.lower()
    
    text = ''.join(c for c in review if c not in punctuation)
    
    test_words = text.split()
    
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])
    
    return test_ints

test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[1, 247, 18, 10, 28, 108, 113, 14, 388, 2, 10, 181, 60, 273, 144, 11, 18, 68, 76, 113, 2, 1, 410, 14, 539]]


In [92]:
seq_length = 200

test_features = pad_features(test_ints, seq_length)
print(test_features)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   1 247  18  10  28
  108 113  14 388   2  10 181  60 273 144  11  18  68  76 113   2   1 410
   14 539]]


In [93]:
test_feature_tensor = torch.from_numpy(test_features)
print(test_feature_tensor.size())

torch.Size([1, 200])


In [109]:
def predict(net, review, seq_length=200):
    
    net.eval()
    
    test_ints = tokenize_review(review)
    
    seq_length=seq_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    h = net.init_hidden(batch_size)
    
    feature_tensor = feature_tensor.to(device)
    
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [110]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [112]:
# call function
seq_length=200 # good to use the length that was trained on

predict(net, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.983161
Positive review detected!
